In [1]:
import os
import sys
from langchain.document_loaders.json_loader import JSONLoader
from langchain.docstore.document import Document
import json
import re
from langchain.vectorstores import FAISS
from langchain.embeddings import BedrockEmbeddings
from functools import reduce
from langchain.prompts import PromptTemplate
from sqlalchemy import MetaData
from sqlalchemy import create_engine


import re
import pandas as pd
import numpy as np
import json
import sqlite3

# Rag 

In [23]:
# Find all items in df  with plot value longer than 512 characters
def find_long_plot_items(df):
    long_plot_items = df[df["text"].str.len() > 512]
    return long_plot_items


def create_text(row, max_len=509):
    text = ""
    for col, val in row.items():
        text += f"{col}: {val},"
    if len(text) > max_len:
        text = text[:max_len] + "..."

    # print(text.rstrip("\n"))
    return text.rstrip()


In [25]:
files = os.listdir('./data/rag')
df = pd.DataFrame()

for f_name in files:
    with open(f'./data/rag/{f_name}', 'rb') as ofp:
        df_tmp = pd.DataFrame(json.load(ofp))
        df = pd.concat([df, df_tmp])

df = df[~df['sql_query'].isnull()]

# Assuming your DataFrame is called 'df'
df["text"] = df.apply(create_text, axis=1)
df = df.drop_duplicates(subset=['question'])
find_long_plot_items(df).count()

sql_query      0
tableName      0
question       0
tableSchema    0
text           0
dtype: int64

In [5]:
import boto3, json
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs


region_name = "us-west-2"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]
aos_host

'search-opensearch-workshop-jfzxbguc2vaocvfkupv5drv4ei.us-west-2.es.amazonaws.com'

In [26]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

In [27]:
import requests

search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

response = requests.get(
    "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
)
model_info = json.loads(response.text)

model_id = model_info["hits"]["hits"][0]["_id"]
model_id

'uRHobJABd4TBsYJJDHfj'

In [28]:
pipeline = {
    "description": "An neural search pipeline for movie index - OpenSearch-cohere-060124084807",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "text": "vector_field",
                },
            }
        }
    ],
}

pipeline_id = "text2sql_plot"
# aos_client.ingest.delete_pipeline(id=pipeline_id)
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

In [14]:
index_name = "rag_semantic_ver2"

In [13]:
# aos_client.indices.delete(index=index_name)

{'acknowledged': True}

In [15]:
rag_semantic = {
    "settings": {
        "max_result_window": 15000,
        # "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "l2",
    },
    "mappings": {
        "properties": {
            "sql_query": {
                "type": "text",
                "fields": {
                          "english": {
                            "type": "text",
                            "analyzer": "english"},                
                            },
            },            
            "tableName": {
                "type": "text",
                "fields": {
                          "english": {
                            "type": "text",
                            "analyzer": "english"},                
                            },
            },
            "question": {
                "type": "text",
                "fields": {
                          "english": {
                            "type": "text",
                            "analyzer": "english"},                
                            },                
            },
            "tableSchema": {
                "type": "text",
                "fields": {
                          "english": {
                            "type": "text",
                            "analyzer": "english"},                
                            },     #{"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },

        }
    },
}



aos_client.indices.create(index=index_name, body=rag_semantic)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'rag_semantic_ver2'}

In [29]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=10, thread_count=1, queue_size=1
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

In [30]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

{'count': 102, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [31]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["vector_field"]},
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["sql_query", "tableName", "question", "tableSchema"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["sql_query"],            
            hit["_source"]["tableName"],
            hit["_source"]["question"],
            hit["_source"]["tableSchema"],            
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score","sql_query", "tableName", "question", "tableSchema"]
    )
    return query_result_df

In [32]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["sql_query"],            
            hit["_source"]["tableName"],
            hit["_source"]["question"],
            hit["_source"]["tableSchema"],            
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score","sql_query", "tableName", "question", "tableSchema"]
    )
    return query_result_df

In [33]:
def hybrid_search(query_text, keyword_weight=0.3, semantic_weight=0.7):
    query = {
        "size": 10,
        "_source": {"exclude": ["text", "vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                        "fields": ["sql_query", "tableName", "question", "tableSchema"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": model_id,
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [keyword_weight, semantic_weight]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["sql_query"],            
            hit["_source"]["tableName"],
            hit["_source"]["question"],
            hit["_source"]["tableSchema"],            
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score","sql_query", "tableName", "question", "tableSchema"]
    )
    return query_result_df

In [34]:
query_text = "people email"

display(keyword_search(query_text))

display(semantic_search(query_text))

display(hybrid_search(query_text, keyword_weight=0.1, semantic_weight=0.9))

,_score,sql_query,tableName,question,tableSchema
0,2.120437,"SELECT T1.customer_name , T1.customer_phone ,...",Customers,"What is the name, phone number, and email addr...",customer_id|payment_method_code|customer_code|...
1,2.120437,"SELECT T1.customer_name , T1.customer_phone ,...",Customers,"What is the name, phone number, and email addr...",customer_id|payment_method_code|customer_code|...


,_score,sql_query,tableName,question,tableSchema
0,0.404748,"SELECT T1.customer_name , T1.customer_phone ,...",Customers,"What is the name, phone number, and email addr...",customer_id|payment_method_code|customer_code|...
1,0.404525,"SELECT T1.customer_name , T1.customer_phone ,...",Customers,"What is the name, phone number, and email addr...",customer_id|payment_method_code|customer_code|...
2,0.393712,SELECT customer_phone FROM customers UNION SEL...,Customers,"""What are all the unique phone numbers present...",customer_id|payment_method_code|customer_code|...
3,0.393712,SELECT customer_phone FROM customers UNION SEL...,Customers,"""What are all the unique phone numbers present...",customer_id|payment_method_code|customer_code|...
4,0.391862,"SELECT customer_name , customer_id FROM custo...",Customers,What are the names and IDs of customers whose ...,customer_id|payment_method_code|customer_code|...
5,0.391651,"SELECT customer_name , customer_id FROM custo...",Customers,What are the names and IDs of customers whose ...,customer_id|payment_method_code|customer_code|...
6,0.390582,"SELECT payment_method_code , count(*) FROM cu...",Customers,"""How many customers are there for each payment...",customer_id|payment_method_code|customer_code|...
7,0.390582,"SELECT payment_method_code , count(*) FROM cu...",Customers,"""How many customers are there for each payment...",customer_id|payment_method_code|customer_code|...
8,0.390182,"SELECT T1.supplier_name , T1.supplier_phone F...",Suppliers,What are the names and phone numbers of suppli...,supplier_id|supplier_name|supplier_phone
9,0.390024,"SELECT customer_id , customer_name FROM custo...",Customers,"""What are the customer IDs and names of custom...",customer_id|payment_method_code|customer_code|...


,_score,sql_query,tableName,question,tableSchema
0,1.000000,"SELECT T1.customer_name , T1.customer_phone ,...",Customers,"What is the name, phone number, and email addr...",customer_id|payment_method_code|customer_code|...
1,0.995654,"SELECT T1.customer_name , T1.customer_phone ,...",Customers,"What is the name, phone number, and email addr...",customer_id|payment_method_code|customer_code|...
2,0.685297,SELECT customer_phone FROM customers UNION SEL...,Customers,"""What are all the unique phone numbers present...",customer_id|payment_method_code|customer_code|...
3,0.685297,SELECT customer_phone FROM customers UNION SEL...,Customers,"""What are all the unique phone numbers present...",customer_id|payment_method_code|customer_code|...
4,0.649302,"SELECT customer_name , customer_id FROM custo...",Customers,What are the names and IDs of customers whose ...,customer_id|payment_method_code|customer_code|...
5,0.645212,"SELECT customer_name , customer_id FROM custo...",Customers,What are the names and IDs of customers whose ...,customer_id|payment_method_code|customer_code|...
6,0.624408,"SELECT payment_method_code , count(*) FROM cu...",Customers,"""How many customers are there for each payment...",customer_id|payment_method_code|customer_code|...
7,0.624408,"SELECT payment_method_code , count(*) FROM cu...",Customers,"""How many customers are there for each payment...",customer_id|payment_method_code|customer_code|...
8,0.616627,"SELECT T1.supplier_name , T1.supplier_phone F...",Suppliers,What are the names and phone numbers of suppli...,supplier_id|supplier_name|supplier_phone
9,0.613546,"SELECT customer_id , customer_name FROM custo...",Customers,"""What are the customer IDs and names of custom...",customer_id|payment_method_code|customer_code|...


In [37]:
def merge_total_meta_rag(rag_table, meta):
    meta = meta[meta['table_name'].isin(rag_table)]

    meta['order'] = pd.Categorical(meta['table_name'], categories=rag_table, ordered=True)
    meta = meta.sort_values(by=['order'])
    meta.reset_index(inplace=True, drop = True)
    meta = meta.drop(columns=['order'])   
    return meta

# SQL DDL 생성 함수
def generate_sql_ddl(df):
    ddl_dict = {}
    for _, row in df.iterrows():
        table_name = row['table_name']
        col_name = row['col_name']
        col_type = row['type']
        if col_type == 'numbe':  # Fixing the typo in 'number'
            col_type = 'number'
        if table_name not in ddl_dict:
            ddl_dict[table_name] = []
        ddl_dict[table_name].append(f'    {col_name} {col_type.upper()}')
    
    ddl_statements = []
    for table_name, columns in ddl_dict.items():
        columns_str = ",\n".join(columns)
        ddl = f'CREATE TABLE {table_name} (\n{columns_str}\n);'
        ddl_statements.append(ddl)
    
    return "\n\n".join(ddl_statements)

In [38]:
rag_df = hybrid_search(query_text, keyword_weight=0.1, semantic_weight=0.9)
fewshot_df = rag_df.drop_duplicates(subset=['question', 'sql_query'])
rag_table = rag_df['tableName'].drop_duplicates(keep='first')[:5].values
meta = pd.read_csv('./data/total_meta.csv')

meta = merge_total_meta_rag(rag_table, meta)

# SQL DDL 생성 예시
sql_ddl = generate_sql_ddl(meta)

/tmp/ipykernel_8805/1673983867.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta['order'] = pd.Categorical(meta['table_name'], categories=rag_table, ordered=True)


# Text 2 SQL 시작

In [39]:
import boto3
import json

from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_aws import ChatBedrock
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

In [66]:
### Connect DB & DB Info

db = SQLDatabase.from_uri("sqlite:///data/department_store.sqlite", sample_rows_in_table_info=3)
db.get_usable_table_names()

db.run("SELECT * FROM Customers LIMIT 10;")

context = db.get_context()

region_name = "us-west-2"

model_kwargs = {  # anthropic
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2048,
    "temperature": 0,
    "stop_sequences": ["\n\nHuman"]
}

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # 파운데이션 모델 지정
    model_kwargs=model_kwargs,
    region_name=region_name,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)  # Claude 속성 구성

## Fewshot

In [72]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


class SQLOutput(BaseModel):
    question: str = Field(description="User Question")
    sql_query: str = Field(description="SQL Query to run")
    
output_parser = JsonOutputParser(pydantic_object=SQLOutput)

In [73]:
examples = []
for idx, x in fewshot_df.iterrows():
    examples.append({'input':x['question'], 'query':x['sql_query']})

In [80]:
prefix_template = """
You are a SQLite expert. Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Below are a number of examples of questions and their corresponding SQL queries.
"""

suffix_template = """
User input: {input}
SQL query: 

{output_format_instruction}
SQL query part should be written in one line.

If you don't have any appropriate answer, just return json with empty string values.

Skip the preamble and go straight into json
"""

example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix=prefix_template,
    suffix=suffix_template,
    input_variables=["input", "top_k", "table_info", "dialect"],
    partial_variables={"output_format_instruction": output_parser.get_format_instructions()}
)

In [70]:
examples

[{'input': 'What is the name, phone number, and email address of the customer who has placed the most orders?',
  'query': 'SELECT T1.customer_name ,  T1.customer_phone ,  T1.customer_email FROM customers AS T1 JOIN customer_orders AS T2 ON T1.customer_id  =  T2.customer_id GROUP BY T2.customer_id ORDER BY count(*) DESC LIMIT 1'},
 {'input': 'What are the names and phone numbers of suppliers, ordered by their address details?',
  'query': 'SELECT T1.supplier_name ,  T1.supplier_phone FROM Suppliers AS T1 JOIN supplier_addresses AS T2 ON T1.supplier_id  =  T2.supplier_id JOIN addresses AS T3 ON T2.address_id  =  T3.address_id ORDER BY T3.address_details'},
 {'input': 'What are the names and IDs of customers whose addresses are located in Tennessee?',
  'query': 'SELECT customer_name ,  customer_id FROM customers WHERE customer_address LIKE "%TN%"'},
 {'input': '"How many customers are there for each payment method?"',
  'query': 'SELECT payment_method_code ,  count(*) FROM customers GRO

In [ ]:
examples = [
    {
        "input": "What are the ids of the top three products that were purchased in the largest amount?", 
         "query": "SELECT product_id FROM product_suppliers ORDER BY total_amount_purchased DESC LIMIT 3"
    },
    {
        "input": "Give the ids of the three products purchased in the largest amounts.",
        "query": "SELECT product_id FROM product_suppliers ORDER BY total_amount_purchased DESC LIMIT 3"
    },
    {
        "input": "What are the product id and product type of the cheapest product?",
        "query": "SELECT product_id ,  product_type_code FROM products ORDER BY product_price LIMIT 1"
    },
    {
        "input": "Give the id and product type of the product with the lowest price.",
        "query": "SELECT product_id ,  product_type_code FROM products ORDER BY product_price LIMIT 1",
    },
    {
        "input": "Find the number of different product types.",
        "query": "SELECT count(DISTINCT product_type_code) FROM products"
    }
]

In [81]:
write_query = create_sql_query_chain(llm, db, prompt)
write_query.get_prompts()[0].pretty_print()


You are a SQLite expert. Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double 

In [82]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

question = "How many customers are there?"

chain = write_query | output_parser
result = chain.invoke({"question": question})

query_result = db.run(result["sql_query"])
query_result

{"question": "How many customers are there?", "sql_query": "SELECT count(*) FROM customers"}

'[(15,)]'


## Pred

In [83]:
## 전체 테스트 데이터에서 target db 관련된것만 추출 
target_db = 'department_store'
with open('./evaluation/train_data/train_spider.json', 'rb') as ofp:
    train = json.load(ofp)

res = []
for t in train:
    if t['db_id'] == target_db:
        res.append((t['question'], t['query']))


# question file 저장할 파일
question_path = './evaluation/question.txt'

# answer file  저장할 파일
answer_path = './evaluation/answer.txt'

# 파일1과 파일2에 데이터를 쓰는 함수
def write_to_files(data, file1_path, file2_path):
    with open(file1_path, 'w') as file1, open(file2_path, 'w') as file2:
        for item in data:
            file1.write(item[0] + '\n')
            file2.write(item[1]+f'\t{target_db}' + '\n')

# 함수 호출하여 파일 생성
write_to_files(res, question_path, answer_path)

print(f"Data has been written to {question_path} and {answer_path}")


Data has been written to ./evaluation/question.txt and ./evaluation/answer.txt


In [85]:
import re

def normalize_sql(query):
    # Remove double quotes from column and table names
    query = re.sub(r'"(\w+)"', r'\1', query)
    # Normalize whitespace
    query = re.sub(r'\s+', ' ', query).strip()
    return query



# question file 저장할 파일
question_path = './evaluation/question.txt'

# answer file  저장할 파일
answer_path = './evaluation/answer.txt'

with open(question_path) as f:
    qlist = [l.strip().split('\t')[0] for l in f.readlines() if len(l.strip()) > 0]


test = False
file_prefix = 'All_leakage'
if test:
    qlist = qlist[:5]
    

In [86]:
pred = []

for q in qlist:
    print('question===>', q)
    p = chain.invoke({"question": q})
    pred.append(normalize_sql(p['sql_query']))

## save file

pred_path = f'./evaluation/{file_prefix}_pred.txt'
print(pred_path)
with open(pred_path, 'w') as file1:
    for item in pred:
        file1.write(item + '\n')


question===> What are the ids of the top three products that were purchased in the largest amount?
{"question": "What are the ids of the top three products that were purchased in the largest amount?", "sql_query": "SELECT \"product_id\" FROM \"Product_Suppliers\" GROUP BY \"product_id\" ORDER BY SUM(\"total_amount_purchased\") DESC LIMIT 3;"}question===> Give the ids of the three products purchased in the largest amounts.
{"question": "Give the ids of the three products purchased in the largest amounts.", "sql_query": "SELECT \"product_id\" FROM \"Product_Suppliers\" GROUP BY \"product_id\" ORDER BY SUM(\"total_amount_purchased\") DESC LIMIT 3;"}question===> What are the product id and product type of the cheapest product?
{"question": "What are the product id and product type of the cheapest product?", "sql_query": "SELECT \"product_id\", \"product_type_code\" FROM \"Products\" ORDER BY \"product_price\" ASC LIMIT 1;"}question===> Give the id and product type of the product with the l

{"question": "What are the distinct names of customers with an order status of Pending, sorted by customer id?", "sql_query": "SELECT DISTINCT \"customer_name\" FROM \"Customers\" JOIN \"Customer_Orders\" ON \"Customers\".\"customer_id\" = \"Customer_Orders\".\"customer_id\" WHERE \"order_status_code\" = 'Pending' ORDER BY \"customer_id\""}question===> Find the name and address of the customers who have both New and Pending orders.
{"question": "Find the name and address of the customers who have both New and Pending orders.", "sql_query": "SELECT c.customer_name, c.customer_address FROM Customers c JOIN Customer_Orders co ON c.customer_id = co.customer_id WHERE co.customer_id IN (SELECT customer_id FROM Customer_Orders WHERE order_status_code = 'New') AND co.customer_id IN (SELECT customer_id FROM Customer_Orders WHERE order_status_code = 'Pending') GROUP BY c.customer_id;"}question===> What are the names and addressed of customers who have both New and Pending orders?
{"question": "W

{"question": "What are the distinct ids of customers who made an order after any order that was Cancelled?", "sql_query": "SELECT DISTINCT T1.customer_id FROM Customer_Orders T1 JOIN Customer_Orders T2 ON T1.order_date > T2.order_date AND T2.order_status_code = 'Cancelled'"}question===> What is id of the staff who had a Staff Department Assignment earlier than any Clerical Staff?
{"question": "What is id of the staff who had a Staff Department Assignment earlier than any Clerical Staff?", "sql_query": "SELECT staff_id FROM staff_department_assignments WHERE date_assigned_from < (SELECT min(date_assigned_from) FROM staff_department_assignments WHERE job_title_code = 'Clerical Staff') ORDER BY date_assigned_from LIMIT 1;"}question===> Return the id of the staff whose Staff Department Assignment was earlier than that of any Clerical Staff.
{"question": "Return the id of the staff whose Staff Department Assignment was earlier than that of any Clerical Staff.", "sql_query": "SELECT staff_id

{"question": "Return the average total amount purchased and total value purchased for the supplier who supplies the greatest number of products.", "sql_query": "SELECT AVG(total_amount_purchased) AS avg_total_amount_purchased, AVG(total_value_purchased) AS avg_total_value_purchased FROM Product_Suppliers WHERE supplier_id = (SELECT supplier_id FROM Product_Suppliers GROUP BY supplier_id ORDER BY COUNT(DISTINCT product_id) DESC LIMIT 1);"}question===> What is the largest and smallest customer codes?
{"question": "What is the largest and smallest customer codes?", "sql_query": "SELECT MAX(\"customer_code\") AS largest_code, MIN(\"customer_code\") AS smallest_code FROM \"Customers\""}question===> Return the maximum and minimum customer codes.
{"question": "Return the maximum and minimum customer codes.", "sql_query": "SELECT max(\"customer_code\") AS max_customer_code, min(\"customer_code\") AS min_customer_code FROM \"Customers\""}question===> List the names of all the distinct customers

{"question": "Find the average price of all product clothes.", "sql_query": "SELECT AVG(\"product_price\") FROM \"Products\" WHERE \"product_type_code\" = 'Clothes'"}question===> What is the average price of clothes?
{"question": "What is the average price of clothes?", "sql_query": "SELECT AVG(\"product_price\") FROM \"Products\" WHERE \"product_type_code\" = 'Clothes'"}question===> Find the name of the most expensive hardware product.
{"question": "Find the name of the most expensive hardware product.", "sql_query": "SELECT \"product_name\" FROM \"Products\" ORDER BY \"product_price\" DESC LIMIT 1;"}question===> What is the name of the hardware product with the greatest price?
{"question": "What is the name of the hardware product with the greatest price?", "sql_query": "SELECT \"product_name\" FROM \"Products\" WHERE \"product_type_code\" = 'Hardware' ORDER BY \"product_price\" DESC LIMIT 1;"}./evaluation/All_leakage_pred.txt


In [87]:
import subprocess

# Python 파일 실행 명령어
command = f'''python evaluation.py --gold '{answer_path}' --pred '{pred_path}' --db './data/database' --table './data/tables.json' --etype exec '''

print(command)
# subprocess 모듈을 사용하여 명령어 실행
result = subprocess.run(command, shell=True, capture_output=True, text=True)

# 출력 결과 확인
print(result.stdout)
print(result.stderr)


python evaluation.py --gold './evaluation/answer.txt' --pred './evaluation/All_leakage_pred.txt' --db './data/database' --table './tables.json' --etype exec 
[pred]
[(4,), (8,), (5,)]
SELECT product_id FROM Product_Suppliers GROUP BY product_id ORDER BY SUM(total_amount_purchased) DESC LIMIT 3;
[gold]
[(4,), (5,), (8,)]
SELECT product_id FROM product_suppliers ORDER BY total_amount_purchased DESC LIMIT 3
eval_err_num:1
------------------
[pred]
[(4,), (8,), (5,)]
SELECT product_id FROM Product_Suppliers GROUP BY product_id ORDER BY SUM(total_amount_purchased) DESC LIMIT 3;
[gold]
[(4,), (5,), (8,)]
SELECT product_id FROM product_suppliers ORDER BY total_amount_purchased DESC LIMIT 3
eval_err_num:2
------------------
[pred]
[('7308 Joan Lake Suite 346\nLizethtown, DE 56522',)]
SELECT customer_address FROM Customers WHERE customer_id = 10;
[gold]
[("36594 O'Keefe Lock\nNew Cali, RI 42319",)]
SELECT T1.address_details FROM addresses AS T1 JOIN customer_addresses AS T2 ON T1.address_id  = 